# EDA : E-COMMERCE CUSTOMER BEHAVIOR

## DATA PREPARATION

### PANDAS, NUMPY, MATPLOTLIB AND PLOTLY INITIATION 

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

### PYSPARK INITIATION 

In [ ]:
import findspark
findspark.init()

In [2]:
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.3.0-bin-hadoop3'

In [66]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### READ THE DATASET FORM LOCALFILES (CSV)

In [4]:
df_nov = spark.read.option('header','true').csv('/Users/Febri/Desktop/DATA ANALYTICS/ecommerce/archive/2019-Nov.csv',inferSchema=True)

In [5]:
df_nov.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
#How many rows on dataframe
df_nov.count()

67501979

In [7]:
#how many columns on dataframe
len(df_nov.columns)

9

In [8]:
df_nov.show(5)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 07:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 07:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 07:00:01|      view|  17302664|2053013553853497655|                null| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 07:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 07:00:01|      view|   1004775|2053013555631882655|electronics.smart...|xiaomi

From the dipslay above, i have 9 columns dan 67501979 rows for my dataframe called df_nov. There are 9 columns consisting of event time, event type, product id, category id, category code, brand, price, user id, and user session.

## DATA CLEANING

In [67]:
#import sql fucntions from pyspark
from pyspark.sql.functions import *

In [68]:
#split category_code columns to more spesific columns(category, sub_category, product)
df_new = df_nov.withColumn('category', split(df_nov['category_code'], '\.').getItem(0)) \
        .withColumn('sub_category', split(df_nov['category_code'], '\.').getItem(1)) \
        .withColumn('product', split(df_nov['category_code'], '\.').getItem(2)).drop('category_code')

In [11]:
df_new = df_new.select("event_time","event_type","product_id","category_id","category","sub_category","product","brand","price","user_id","user_session")

In [69]:
#drop the null/na value from event_type, category, sub_category, product, brand, and price columns with treshold = 4
df_new = df_new.dropna(subset=["event_type","category","sub_category","product","brand","price"], thresh=4)
df_new.show(5)

+-------------------+----------+----------+-------------------+------+------+---------+--------------------+-----------+--------------+-------+
|         event_time|event_type|product_id|        category_id| brand| price|  user_id|        user_session|   category|  sub_category|product|
+-------------------+----------+----------+-------------------+------+------+---------+--------------------+-----------+--------------+-------+
|2019-11-01 07:00:00|      view|   1003461|2053013555631882655|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|electronics|    smartphone|   null|
|2019-11-01 07:00:00|      view|   5000088|2053013566100866035|janome|293.65|530496790|8e5f4f83-366c-4f7...| appliances|sewing_machine|   null|
|2019-11-01 07:00:01|      view|   3601530|2053013563810775923|    lg|712.87|518085591|3bfb58cd-7892-48c...| appliances|       kitchen| washer|
|2019-11-01 07:00:01|      view|   1004775|2053013555631882655|xiaomi|183.27|558856683|313628f1-68b8-460...|electronics|    smartphone| 

In [13]:
#ROWS ON CLEAN DATAFRAME
df_new.count()

45603808

In [14]:
#COLUMNS ON CLEAN DATAFRAME
len(df_new.columns)

11

In [15]:
df_new.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



Before i start to explore the data, i make the data more clean. This is must be done to get more precise results and reduce bias that could occur in the insights obtained. Now, i hace clean dataframe called df_new with 11 columns there are event_time, event_type, product_id, category_id, category, sub_category, product, brand, price, user_id, and user session. For rows, i have 45603808 active rows and drop the rest because contain null/na value.

## DATA EXPLORATION

### HOW MANY EACH EVENT TYPE THAT HAPPENED?

In [82]:
data = [
        ("view",df_new.filter(df_new['event_type']=='view').count()),
        ("cart",df_new.filter(df_new['event_type']=='cart').count()),
        ("remove from cart",df_new.filter(df_new['event_type']=='remove_from_cart').count()),
        ("purchase",df_new.filter(df_new['event_type']=='purchase').count())
        ]

In [83]:
columns = ["event_type","total_event"]
df_et = spark.createDataFrame(data,columns)
df_et.show()

+----------------+-----------+
|      event_type|total_event|
+----------------+-----------+
|            view|   42718650|
|            cart|    2202437|
|remove from cart|          0|
|        purchase|     682721|
+----------------+-----------+



In [112]:
#export pyspark dataframe to pandas dataframe format
df_etP = df_et.toPandas()

In [123]:
#make bar chart using plotly express 
bar_etP = px.bar(x = df_etP.event_type,
                 y = df_etP.total_event,
                 title = 'E-COMMERCE CUSTOMER BEHAVIOR BASED ON EVENT TYPE')

bar_etP.update_layout(xaxis_title = 'EVENT TYPE',
                  yaxis_title = 'TOTAL')
bar_etP.show()

On the section code above, i make new df called df_et. df_et contains total event that happened. We can see from the table and bar chart, customers have a tendency to view more than pick some item into the cart and purchase it. But, the good thing customers never remove their stuff from their cart.

In [116]:
#drop row remove_from_cart 
df_etd = df_et.where("total_event!=0")
df_etd.show()

+----------+-----------+
|event_type|total_event|
+----------+-----------+
|      view|   42718650|
|      cart|    2202437|
|  purchase|     682721|
+----------+-----------+



In [120]:
#validating number of event that happened
df_etd.agg({'total_event':'sum'}).collect()

[Row(sum(total_event)=45603808)]

### HOW MANY EVENT TYPE BASED ON CATEGORY?

In [132]:
df_view_cat = df_new.where(df_new.event_type=="view").groupBy('event_type','category').count().withColumnRenamed('count','total')
df_view_cat.sort(col('total').desc()).show(5)

+----------+-----------+--------+
|event_type|   category|   total|
+----------+-----------+--------+
|      view|electronics|21988192|
|      view| appliances| 8037980|
|      view|  computers| 4029049|
|      view|    apparel| 2942622|
|      view|  furniture| 2058798|
+----------+-----------+--------+
only showing top 5 rows



In [133]:
df_view_catP = df_view_cat.toPandas()

In [162]:
bar_view_catP = px.bar(x = df_view_catP.category,
                       y = df_view_catP.total,
                       text = df_view_catP.total,
                       color= df_view_catP.total,
                       title = 'VIEW EVENT BY CUSTOMERS')

bar_view_catP.update_layout(xaxis_title = 'CATEGORY',
                            yaxis_title = 'TOTAL',
                            xaxis = {'categoryorder':'total descending'})

bar_view_catP.update_traces(textposition='outside')

bar_view_catP.show()

In [130]:
#validating total view event 
df_view_cat.agg({'count':'sum'}).collect()

[Row(sum(count)=42718650)]

On the section above, there are dataframe called df_view_cat that contains view event based on category. Here we can see most customer like to view electronics product, appliances product, computers product, etc. Number of view by customers on the electronics products is very big (2198812) , almost three times from appliances product (8037980). Or i can said the demand for electronic products is very big.

In [155]:
df_cart_cat = df_new.where(df_new.event_type=="cart").groupBy('event_type','category').count().withColumnRenamed('count','total')
df_cart_cat.sort(col('count').desc()).show(5)

+----------+-----------+-------+
|event_type|   category|  total|
+----------+-----------+-------+
|      cart|electronics|1524255|
|      cart| appliances| 352819|
|      cart|  computers| 117832|
|      cart|    apparel|  54264|
|      cart|  furniture|  41489|
+----------+-----------+-------+
only showing top 5 rows



In [156]:
df_cart_catP = df_cart_cat.toPandas()

In [163]:
bar_cart_catP = px.bar(x = df_cart_catP.category,
                       y = df_cart_catP.total,
                       text = df_cart_catP.total,
                       color = df_cart_catP.total,
                       title = 'CART EVENT BY CUSTOMERS')

bar_cart_catP.update_layout(xaxis_title = 'CATEGORY',
                            yaxis_title = 'TOTAL',
                            xaxis = {'categoryorder':'total descending'})

bar_cart_catP.update_traces(textposition='outside')

bar_cart_catP.show()

In [27]:
#validating total cart event
df_cart_cat.agg({'count':'sum'}).collect()

[Row(sum(count)=2202437)]

From the cart event, electronic products also have the largest number, there is 1524255.

In [158]:
df_remove_cat = df_new.where(df_new.event_type=="remove_from_cart").groupBy('event_type','category').count().withColumnRenamed('count','total')
df_remove_cat.sort(col('count').desc()).show(5)

+----------+--------+-----+
|event_type|category|total|
+----------+--------+-----+
+----------+--------+-----+



In [159]:
df_purchase_cat = df_new.where(df_new.event_type=="purchase").groupBy('event_type','category').count().withColumnRenamed('count','total')
df_purchase_cat.sort(col('count').desc()).show(5)

+----------+-----------+------+
|event_type|   category| total|
+----------+-----------+------+
|  purchase|electronics|493639|
|  purchase| appliances| 99026|
|  purchase|  computers| 34477|
|  purchase|    apparel| 14215|
|  purchase|  furniture| 11542|
+----------+-----------+------+
only showing top 5 rows



In [164]:
df_purchase_catP = df_purchase_cat.toPandas()

In [165]:
bar_purchase_catP = px.bar(x = df_purchase_catP.category,
                       y = df_purchase_catP.total,
                       text = df_purchase_catP.total,
                       color = df_purchase_catP.total,
                       title = 'PURCHASE EVENT BY CUSTOMERS')

bar_purchase_catP.update_layout(xaxis_title = 'CATEGORY',
                            yaxis_title = 'TOTAL',
                            xaxis = {'categoryorder':'total descending'})

bar_purchase_catP.update_traces(textposition='outside')

bar_purchase_catP.show()

In [30]:
##validating total purchase event
df_purchase_cat.agg({'count':'sum'}).collect()

[Row(sum(count)=682721)]

From the data, we can conclude that customers have a high interest in the electronics category. Both from the number of views, carts or purchases. Electronics is in first place

In [31]:
df_cat = df_view_cat.unionByName(df_cart_cat).unionByName(df_purchase_cat)
df_cat.show(5)

+----------+-----------+-------+
|event_type|   category|  count|
+----------+-----------+-------+
|      view|  furniture|2058798|
|      view|       auto|1164278|
|      view| stationery|  11943|
|      view|accessories| 380373|
|      view|       kids| 779517|
+----------+-----------+-------+
only showing top 5 rows



In [166]:
df_catP = df_cat.toPandas()

In [214]:
bar_catP = px.bar(x = df_catP.category,
                  y = df_catP.total,
                  text = df_catP.total,
                  color = df_catP.event_type,
                  barmode = 'group',
                  title = 'CATEGORY BASED ON EVENT TYPE BY CUSTOMERS')

bar_catP.update_layout(xaxis_title = 'CATEGORY',
                       yaxis_title = 'TOTAL',
                       xaxis = {'categoryorder':'total descending'})

bar_catP.update_traces(textposition='inside')

bar_catP.show()

I make a group bar chart so you can see category based on event type trends easily.

In [32]:
#renamed 'count' column with 'total'
df_cat = df_cat.withColumnRenamed('count','total')
df_cat.show(5)

+----------+-----------+-------+
|event_type|   category|  total|
+----------+-----------+-------+
|      view|  furniture|2058798|
|      view|       auto|1164278|
|      view| stationery|  11943|
|      view|accessories| 380373|
|      view|       kids| 779517|
+----------+-----------+-------+
only showing top 5 rows



In [33]:
#validating total of event type based on category
df_cat.agg({'total':'sum'}).collect()

[Row(sum(total)=45603808)]

### HOW MUCH MONEY COLLECTED EACH BRAND BASED ON EVENT TYPE?

In [189]:
#import decimaltype fucntions for change column type
from pyspark.sql.types import DecimalType

In [35]:
df_pur = df_new.where(df_new.event_type=="purchase").groupBy('event_type','category','sub_category','product','brand').sum('price').withColumnRenamed('sum(price)','total')

In [36]:
df_pur = df_pur.select('event_type','category','sub_category','product','brand',(col('total').cast(DecimalType(38))))
df_pur = df_pur.na.fill("unknown")
df_pur.sort(col('total').desc()).show(5)

+----------+-----------+------------+---------+-------+---------+
|event_type|   category|sub_category|  product|  brand|    total|
+----------+-----------+------------+---------+-------+---------+
|  purchase|electronics|  smartphone|  unknown|  apple|115263068|
|  purchase|electronics|  smartphone|  unknown|samsung| 42710020|
|  purchase|electronics|  smartphone|  unknown| xiaomi|  9844621|
|  purchase|electronics|       video|       tv|samsung|  6406407|
|  purchase|electronics|       audio|headphone|  apple|  4812869|
+----------+-----------+------------+---------+-------+---------+
only showing top 5 rows



In [37]:
df_pur.printSchema()

root
 |-- event_type: string (nullable = false)
 |-- category: string (nullable = false)
 |-- sub_category: string (nullable = false)
 |-- product: string (nullable = false)
 |-- brand: string (nullable = false)
 |-- total: decimal(38,0) (nullable = true)



In [38]:
df_pur.agg({'total':'sum'}).collect()

[Row(sum(total)=Decimal('245314389'))]

Based on purchase history, Apple is number one brand with 115263068 million USD sales.

In [39]:
df_view = df_new.where(df_new.event_type=="view").groupBy('event_type','category','sub_category','product','brand').sum('price').withColumnRenamed('sum(price)','total')

In [40]:
df_view = df_view.select('event_type','category','sub_category','product','brand',(col('total').cast(DecimalType(38))))
df_view = df_view.na.fill("unknown")
df_view.sort(col('total').desc()).show(5)

+----------+-----------+------------+-------+-------+----------+
|event_type|   category|sub_category|product|  brand|     total|
+----------+-----------+------------+-------+-------+----------+
|      view|electronics|  smartphone|unknown|  apple|3939287831|
|      view|electronics|  smartphone|unknown|samsung|1693571583|
|      view|electronics|  smartphone|unknown| xiaomi| 712989978|
|      view|electronics|       video|     tv|samsung| 435820634|
|      view|  computers|    notebook|unknown| lenovo| 331856843|
+----------+-----------+------------+-------+-------+----------+
only showing top 5 rows



In [41]:
df_view.printSchema()

root
 |-- event_type: string (nullable = false)
 |-- category: string (nullable = false)
 |-- sub_category: string (nullable = false)
 |-- product: string (nullable = false)
 |-- brand: string (nullable = false)
 |-- total: decimal(38,0) (nullable = true)



In [42]:
df_view.agg({'total':'sum'}).collect()

[Row(sum(total)=Decimal('14852761181'))]

In [43]:
df_cart = df_new.where(df_new.event_type=="cart").groupBy('event_type','category','sub_category','product','brand').sum('price').withColumnRenamed('sum(price)','total')

In [44]:
df_cart = df_cart.select('event_type','category','sub_category','product','brand',(col('total').cast(DecimalType(38))))
df_cart = df_cart.na.fill("unknown")
df_cart.sort(col('total').desc()).show(5)

+----------+-----------+------------+-------+-------+---------+
|event_type|   category|sub_category|product|  brand|    total|
+----------+-----------+------------+-------+-------+---------+
|      cart|electronics|  smartphone|unknown|  apple|341999985|
|      cart|electronics|  smartphone|unknown|samsung|123060139|
|      cart|electronics|  smartphone|unknown| xiaomi| 39480574|
|      cart|electronics|       video|     tv|samsung| 22199273|
|      cart|electronics|  smartphone|unknown| huawei| 14785089|
+----------+-----------+------------+-------+-------+---------+
only showing top 5 rows



In [45]:
df_cart.printSchema()

root
 |-- event_type: string (nullable = false)
 |-- category: string (nullable = false)
 |-- sub_category: string (nullable = false)
 |-- product: string (nullable = false)
 |-- brand: string (nullable = false)
 |-- total: decimal(38,0) (nullable = true)



In [46]:
df_cart.agg({'total':'sum'}).collect()

[Row(sum(total)=Decimal('775274265'))]

In [47]:
df_smr = df_pur.unionByName(df_view).unionByName(df_cart)
df_smr.show(5)

+----------+-----------+------------+-------------+--------+-----+
|event_type|   category|sub_category|      product|   brand|total|
+----------+-----------+------------+-------------+--------+-----+
|  purchase| appliances|     kitchen|refrigerators|   hansa|42353|
|  purchase|electronics|      clocks|      unknown|  fossil|17948|
|  purchase|  furniture|     kitchen|        chair|     fly|   52|
|  purchase| appliances|     kitchen|          hob|dauscher|22895|
|  purchase|  furniture| living_room|      cabinet|      sv|79672|
+----------+-----------+------------+-------------+--------+-----+
only showing top 5 rows



#### From previous section, we know that electronics is top one in the category. 
#### From three table above which is purchase, view and cart table. 
# We can conclude Apple is the most popular brand. 
#### Even in terms of sales, the brand is able to sell their smartphone products up to  115263068 million USD. 
#### Number 2 is also a giant company, Samsung which has managed to sell their smartphone products up to 42710020 million USD. 
#### And the third is Xiaomi, which managed to sell their smartphone products up to 9844621 million USD.

In [182]:
df_money = df_new.groupBy('event_type').sum('price').withColumnRenamed('sum(price)','total_money')

In [185]:
df_money = df_money.select('event_type',(col('total_money').cast(DecimalType(38)))).withColumnRenamed('event_type','event_type2')
df_money.show()

+-----------+-----------+
|event_type2|total_money|
+-----------+-----------+
|   purchase|  245314384|
|       view|14852761161|
|       cart|  775274240|
+-----------+-----------+



In [186]:
#join 2 dataframe
df_ml = df_etd.join(df_money,df_etd.event_type ==  df_money.event_type2,"left")

In [188]:
df_ml = df_ml.drop('event_type2').show()

+----------+-----------+-----------+
|event_type|total_event|total_money|
+----------+-----------+-----------+
|      view|   42718650|14852761161|
|      cart|    2202437|  775274240|
|  purchase|     682721|  245314384|
+----------+-----------+-----------+



## CONCLUSION

1. From the table, it can be seen that customers can viewing stuff on e-commerce 42718650 times and the total cost of goods is up to 14852761161 billion USD. There are also those who enter the items they want to buy into their basket for a total of 2202437 times, with a total amount of money reaching 775274240 million USD. And customers who finally decided to buy up to 682721 transactions, which has a total of 245314384 million USD.

2. Based on the data above, the most popular products are smartphones from the Apple, Samsung and Xiaomi brands. This is shown from the purchase, view, and cart table.

3. The number of purchases made is far from viewing and adding stuff to their baskets by customers. This can be improveby giving discounts on the electronics category, especially smartphones because in that segment has a high interest value from customers.